UNEMPLOYMENT -  LU_PE_NUM
EMPLOYMENT - LE_PE_NUM
POPULATION - LP_PE_NUM
GDP - NC_GDP_PT

Also, please be aware of the following rate limits and throttle your requests accordingly:

    10 requests in 5 second window from one user (IP)
    50 requests per second overall on the application

In [97]:
import requests
import pprint
import pandas as pd
import config as cfg
import time
import sys
from sqlalchemy import create_engine
from datetime import datetime
from pprint import pprint

In [165]:
def getJsonCountry(index, country):
    key = f'CompactData/IFS/A.{country}.{index}'
    data = requests.get(f'{url}{key}').json()
    return data['CompactData']['DataSet']['Series']

def createDataList(json):
    data = json['Obs']
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    if isinstance(data, list):
        data_list = [(json.get('@REF_AREA'), json.get('@UNIT_MULT'), obs.get('@TIME_PERIOD'), obs.get('@OBS_VALUE'), timestamp) for obs in data]
    else:
        data_list = [(json.get('@REF_AREA'), json.get('@UNIT_MULT'), data.get('@TIME_PERIOD'), data.get('@OBS_VALUE'))]
    
    return data_list

def getDataStructure(dataSet):
    key = f"DataStructure/{dataSet}"
    dimension_list = requests.get(f'{url}{key}').json()
    return dimension_list

def getCountrylistIFS():
    codes = getDataStructure('IFS')["Structure"]["CodeLists"]["CodeList"]
    countries = [(country["@value"], country["Description"]["#text"]) for country in codes[2]['Code']]
    return countries
    

def insertIntoTable(table, tblList):
    tblList = ','.join([str(obs) for obs in tblList])  
    engine.execute("INSERT INTO " + table +" VALUES " + tblList)

def getSqlTableCreateStr(tblName, **fields):
    sqlstr = ', '.join([' '.join([key, str(val)]) for key, val in fields.items()])
    sqlstr = f'CREATE TABLE IF NOT EXISTS {tblName}({sqlstr})' 
    return sqlstr

In [193]:
def insertIntoCountries(countryList, IndexName, secsWait, engine):

    engine.execute(getSqlTableCreateStr(IndexName, REF_AREA='VARCHAR(5)', UNIT_MULT='INT', TIME_PERIOD='INT', VALUE='NUMERIC(18,5)', TIMESTAMP='TIMESTAMP'))

    for country in countryList:
    
        try:
            datacountry = getJsonCountry(IndexName,country[0])
            datacountry = createDataList(datacountry)
            insertIntoTable(IndexName, datacountry)
            print(f"Country {country} exctracted sucessfully for Indicator {IndexName}")
        except:
            print(f"Error ocurred while processing {IndexName} for {country} : {sys.exc_info()[0]}")

        time.sleep(secsWait)


In [194]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
indexes = ['LU_PE_NUM', 'LE_PE_NUM', 'LP_PE_NUM', 'NGDP_XDC']

con_string = 'postgresql://postgres:' + cfg.dbpassword + '@localhost:5432/' + cfg.dbname
engine = create_engine(con_string)
connection = engine.connect()

In [195]:
countries = getCountrylistIFS()

for index in indexes:
    insertIntoCountries(countries, index, 3, engine)


Error ocurred while processing LU_PE_NUM for ('AF', 'Afghanistan') : <class 'KeyError'>
Country ('AL', 'Albania') exctracted sucessfully for Indicator LU_PE_NUM
Country ('DZ', 'Algeria') exctracted sucessfully for Indicator LU_PE_NUM
Error ocurred while processing LU_PE_NUM for ('AO', 'Angola') : <class 'KeyError'>
Error ocurred while processing LU_PE_NUM for ('AI', 'Anguilla') : <class 'KeyError'>
Error ocurred while processing LU_PE_NUM for ('AG', 'Antigua and Barbuda') : <class 'KeyError'>
Error ocurred while processing LU_PE_NUM for ('5M', 'AMF (Arab Monetary Fund)') : <class 'KeyError'>
Country ('AR', 'Argentina') exctracted sucessfully for Indicator LU_PE_NUM
Country ('AM', 'Armenia') exctracted sucessfully for Indicator LU_PE_NUM
Country ('AW', 'Aruba') exctracted sucessfully for Indicator LU_PE_NUM
Country ('AU', 'Australia') exctracted sucessfully for Indicator LU_PE_NUM
Country ('AT', 'Austria') exctracted sucessfully for Indicator LU_PE_NUM
Country ('AZ', 'Azerbaijan') exctr

Country ('IS', 'Iceland') exctracted sucessfully for Indicator LU_PE_NUM
Error ocurred while processing LU_PE_NUM for ('IN', 'India') : <class 'KeyError'>
Country ('ID', 'Indonesia') exctracted sucessfully for Indicator LU_PE_NUM
Error ocurred while processing LU_PE_NUM for ('1E', 'IBRD (International Bank for Reconstruction and Development)') : <class 'KeyError'>
Error ocurred while processing LU_PE_NUM for ('X0', 'IMF Member Countries (IMF)') : <class 'KeyError'>
Country ('IR', 'Iran, Islamic Republic of') exctracted sucessfully for Indicator LU_PE_NUM
Error ocurred while processing LU_PE_NUM for ('IQ', 'Iraq') : <class 'KeyError'>
Country ('IE', 'Ireland') exctracted sucessfully for Indicator LU_PE_NUM
Error ocurred while processing LU_PE_NUM for ('7B', 'IDB (Islamic Development Bank)') : <class 'KeyError'>
Error ocurred while processing LU_PE_NUM for ('IM', 'Isle of Man') : <class 'KeyError'>
Country ('IL', 'Israel') exctracted sucessfully for Indicator LU_PE_NUM
Country ('IT', 'It

Country ('CH', 'Switzerland') exctracted sucessfully for Indicator LU_PE_NUM
Country ('SY', 'Syrian Arab Republic') exctracted sucessfully for Indicator LU_PE_NUM
Error ocurred while processing LU_PE_NUM for ('TW', 'Taiwan, Province of China') : <class 'KeyError'>
Country ('TJ', 'Tajikistan') exctracted sucessfully for Indicator LU_PE_NUM
Country ('TZ', 'Tanzania, United Republic of') exctracted sucessfully for Indicator LU_PE_NUM
Country ('TH', 'Thailand') exctracted sucessfully for Indicator LU_PE_NUM
Country ('TL', 'Timor-Leste') exctracted sucessfully for Indicator LU_PE_NUM
Country ('TG', 'Togo') exctracted sucessfully for Indicator LU_PE_NUM
Error ocurred while processing LU_PE_NUM for ('TO', 'Tonga') : <class 'KeyError'>
Country ('TT', 'Trinidad and Tobago') exctracted sucessfully for Indicator LU_PE_NUM
Country ('TN', 'Tunisia') exctracted sucessfully for Indicator LU_PE_NUM
Country ('TR', 'Turkey') exctracted sucessfully for Indicator LU_PE_NUM
Error ocurred while processing L

Country ('BH', 'Bahrain') exctracted sucessfully for Indicator LE_PE_NUM
Country ('BD', 'Bangladesh') exctracted sucessfully for Indicator LE_PE_NUM
Error ocurred while processing LE_PE_NUM for ('5W', 'BEAC (Banque des Etats de l`Afrique Centrale)') : <class 'KeyError'>
Error ocurred while processing LE_PE_NUM for ('5B', 'BIS (Bank for International Settlements)') : <class 'KeyError'>
Country ('BB', 'Barbados') exctracted sucessfully for Indicator LE_PE_NUM
Country ('BY', 'Belarus') exctracted sucessfully for Indicator LE_PE_NUM
Country ('BE', 'Belgium') exctracted sucessfully for Indicator LE_PE_NUM
Error ocurred while processing LE_PE_NUM for ('R1', 'Belgo-Luxembourg Economic Union') : <class 'KeyError'>
Country ('BZ', 'Belize') exctracted sucessfully for Indicator LE_PE_NUM
Country ('BJ', 'Benin') exctracted sucessfully for Indicator LE_PE_NUM
Country ('BM', 'Bermuda') exctracted sucessfully for Indicator LE_PE_NUM
Country ('BT', 'Bhutan') exctracted sucessfully for Indicator LE_PE_

Country ('JP', 'Japan') exctracted sucessfully for Indicator LE_PE_NUM
Error ocurred while processing LE_PE_NUM for ('JE', 'Jersey') : <class 'KeyError'>
Error ocurred while processing LE_PE_NUM for ('JO', 'Jordan') : <class 'KeyError'>
Country ('KZ', 'Kazakhstan') exctracted sucessfully for Indicator LE_PE_NUM
Country ('KE', 'Kenya') exctracted sucessfully for Indicator LE_PE_NUM
Error ocurred while processing LE_PE_NUM for ('KI', 'Kiribati') : <class 'KeyError'>
Country ('KR', 'Korea, Republic of') exctracted sucessfully for Indicator LE_PE_NUM
Country ('XK', 'Kosovo, Republic of') exctracted sucessfully for Indicator LE_PE_NUM
Error ocurred while processing LE_PE_NUM for ('KW', 'Kuwait') : <class 'KeyError'>
Country ('KG', 'Kyrgyzstan') exctracted sucessfully for Indicator LE_PE_NUM
Country ('LA', 'Lao People`s Democratic Republic') exctracted sucessfully for Indicator LE_PE_NUM
Country ('LV', 'Latvia') exctracted sucessfully for Indicator LE_PE_NUM
Error ocurred while processing LE

...

Country ('MW', 'Malawi') exctracted sucessfully for Indicator LP_PE_NUM
Country ('MY', 'Malaysia') exctracted sucessfully for Indicator LP_PE_NUM
Country ('MV', 'Maldives') exctracted sucessfully for Indicator LP_PE_NUM
Country ('ML', 'Mali') exctracted sucessfully for Indicator LP_PE_NUM
Country ('MT', 'Malta') exctracted sucessfully for Indicator LP_PE_NUM
Country ('MH', 'Marshall islands') exctracted sucessfully for Indicator LP_PE_NUM
Country ('MQ', 'Martinique') exctracted sucessfully for Indicator LP_PE_NUM
Country ('MR', 'Mauritania') exctracted sucessfully for Indicator LP_PE_NUM
Country ('MU', 'Mauritius') exctracted sucessfully for Indicator LP_PE_NUM
Country ('MX', 'Mexico') exctracted sucessfully for Indicator LP_PE_NUM
Country ('FM', 'Micronesia, Federated States of') exctracted sucessfully for Indicator LP_PE_NUM
Country ('MD', 'Moldova, Republic of') exctracted sucessfully for Indicator LP_PE_NUM
Country ('MN', 'Mongolia') exctracted sucessfully for Indicator LP_PE_NUM
C

Error ocurred while processing LP_PE_NUM for ('F19', 'Africa not allocated') : <class 'KeyError'>
Error ocurred while processing LP_PE_NUM for ('1C_ALLC', 'All Countries') : <class 'KeyError'>
Error ocurred while processing LP_PE_NUM for ('1C_ALL', 'All Countries and Country Groups') : <class 'KeyError'>
Error ocurred while processing LP_PE_NUM for ('W0', 'World (all areas, including reference area, including IO)') : <class 'KeyError'>
Error ocurred while processing LP_PE_NUM for ('1C_All_Countries_Published', 'All Countries Published') : <class 'KeyError'>
Error ocurred while processing LP_PE_NUM for ('1C_ALLG', 'All Country Groups') : <class 'KeyError'>
Error ocurred while processing LP_PE_NUM for ('R16', 'APEC (Asia-Pacific Economic Co-operation)') : <class 'KeyError'>
Error ocurred while processing LP_PE_NUM for ('5O', 'BCEAO (Banque Centrale des Etats de l`Afrique de l`Ouest)') : <class 'KeyError'>
Error ocurred while processing LP_PE_NUM for ('5X', 'CEMAC') : <class 'KeyError'>
E

Country ('HR', 'Croatia') exctracted sucessfully for Indicator NGDP_XDC
Error ocurred while processing NGDP_XDC for ('CU', 'Cuba') : <class 'KeyError'>
Error ocurred while processing NGDP_XDC for ('1C_355', 'Curacao & St. Maarten') : <class 'KeyError'>
Error ocurred while processing NGDP_XDC for ('CW', 'Curacao') : <class 'KeyError'>
Country ('CY', 'Cyprus') exctracted sucessfully for Indicator NGDP_XDC
Country ('CZ', 'Czech Republic') exctracted sucessfully for Indicator NGDP_XDC
Error ocurred while processing NGDP_XDC for ('CSH', 'Former Czechoslovakia') : <class 'KeyError'>
Country ('DK', 'Denmark') exctracted sucessfully for Indicator NGDP_XDC
Country ('DJ', 'Djibouti') exctracted sucessfully for Indicator NGDP_XDC
Country ('DM', 'Dominica') exctracted sucessfully for Indicator NGDP_XDC
Country ('DO', 'Dominican Republic') exctracted sucessfully for Indicator NGDP_XDC
Error ocurred while processing NGDP_XDC for ('5I', 'ECCB (Eastern Caribbean Central Bank)') : <class 'KeyError'>
Er

Country ('NZ', 'New Zealand') exctracted sucessfully for Indicator NGDP_XDC
Country ('NI', 'Nicaragua') exctracted sucessfully for Indicator NGDP_XDC
Country ('NE', 'Niger') exctracted sucessfully for Indicator NGDP_XDC
Country ('NG', 'Nigeria') exctracted sucessfully for Indicator NGDP_XDC
Country ('MK', 'North Macedonia, Republic of') exctracted sucessfully for Indicator NGDP_XDC
Country ('NO', 'Norway') exctracted sucessfully for Indicator NGDP_XDC
Country ('OM', 'Oman') exctracted sucessfully for Indicator NGDP_XDC
Country ('PK', 'Pakistan') exctracted sucessfully for Indicator NGDP_XDC
Country ('PW', 'Palau') exctracted sucessfully for Indicator NGDP_XDC
Country ('PA', 'Panama') exctracted sucessfully for Indicator NGDP_XDC
Country ('PG', 'Papua New Guinea') exctracted sucessfully for Indicator NGDP_XDC
Country ('PY', 'Paraguay') exctracted sucessfully for Indicator NGDP_XDC
Country ('PE', 'Peru') exctracted sucessfully for Indicator NGDP_XDC
Country ('PH', 'Philippines') exctract

Error ocurred while processing NGDP_XDC for ('1C_994', 'EPU/EF') : <class 'KeyError'>
Error ocurred while processing NGDP_XDC for ('1C_EMU', '1C_EMU') : <class 'KeyError'>
Error ocurred while processing NGDP_XDC for ('E1', 'Europe') : <class 'KeyError'>
Error ocurred while processing NGDP_XDC for ('1C_080', 'Export earnings: fuel') : <class 'KeyError'>
Error ocurred while processing NGDP_XDC for ('1C_092', 'Export earnings: nonfuel') : <class 'KeyError'>
Error ocurred while processing NGDP_XDC for ('F97', 'Middle East') : <class 'KeyError'>
Error ocurred while processing NGDP_XDC for ('F98', 'Middle East and North Africa (MENA)') : <class 'KeyError'>
Error ocurred while processing NGDP_XDC for ('1C_440', 'Middle East, North Africa, Afghanistan, and Pakistan') : <class 'KeyError'>
Error ocurred while processing NGDP_XDC for ('1C_NANSA', 'National Accounts (NSA) Countries') : <class 'KeyError'>
Error ocurred while processing NGDP_XDC for ('1C_NASA', 'National Accounts (SA) Countries') : 